In [5]:
import numpy as np
import sys
import os
import pandas as pd
from sklearn.metrics import ndcg_score
sys.path.append(os.path.abspath('../src'))
from model_training import get_recommendations, get_content_recommendations_pca, get_sentiment_recommendations, get_recommendations_with_pca, get_review_recommendations, get_topic_recommendations, get_reviewer_overlap_recommendations, get_weighted_hybrid_recommendations, load_all_models, get_content_recommendations, svd_product_recommendations, get_knn_recommendations
# Load the DataFrame and set it in the ml module
from data_preprocessing import main
#main()
#df = pd.read_parquet('../data/processed/amazon_clean.parquet').reset_index(drop=True)
df = pd.read_parquet('../data/processed/amazon_clean.parquet')
print(df.head())
load_all_models()

   product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   
2  B096MSW6CT  Sounce Fast Phone Charging Cable & Data Sync U...   
3  B08HDJ86NZ  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...   
4  B08CF3B7N1  Portronics Konnect L 1.2M Fast Charging 3A 8 P...   

                                            category  discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...             399.0   
1  Computers&Accessories|Accessories&Peripherals|...             199.0   
2  Computers&Accessories|Accessories&Peripherals|...             199.0   
3  Computers&Accessories|Accessories&Peripherals|...             329.0   
4  Computers&Accessories|Accessories&Peripherals|...             154.0   

  actual_price discount_percentage  rating rating_count  \
0       ₹1,099                 64%     4.2       24,269   
1         ₹349              

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:463: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.5.1 when using version 1.8.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:463: InconsistentVersionWarning: Trying to unpickle estimator FunctionTransformer from version 1.5.1 when using version 1.8.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:463: InconsistentVersionWarning: Trying to unpickle estimator ColumnTransformer from version 1.5.1 when using version 1.8.0. This might lead to breaking

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,199.0,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,199.0,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,329.0,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,154.0,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1346,B08L7J3T31,Noir Aqua - 5pcs PP Spun Filter + 1 Spanner | ...,Home&Kitchen|Kitchen&HomeAppliances|WaterPurif...,379.0,₹919,59%,4.0,"1,090",SUPREME QUALITY 90 GRAM 3 LAYER THIK PP SPUN F...,"AHITFY6AHALOFOHOZEOC6XBP4FEA,AFRABBODZJZQB6Z4U...","Prabha ds,Raghuram bk,Real Deal,Amazon Custome...","R3G3XFHPBFF0E8,R3C0BZCD32EIGW,R2EBVBCN9QPD9R,R...","Received the product without spanner,Excellent...","I received product without spanner,Excellent p...",https://m.media-amazon.com/images/I/41fDdRtjfx...,https://www.amazon.in/Noir-Aqua-Spanner-Purifi...
1347,B01M6453MB,Prestige Delight PRWO Electric Rice Cooker (1 ...,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,2280.0,"₹3,045",25%,4.1,"4,118","230 Volts, 400 watts, 1 Year","AFG5FM3NEMOL6BNFRV2NK5FNJCHQ,AGEINTRN6Z563RMLH...","Manu Bhai,Naveenpittu,Evatira Sangma,JAGANNADH...","R3DDL2UPKQ2CK9,R2SYYU1OATVIU5,R1VM993161IYRW,R...","ok,everything was good couldn't return bcoz I ...","ok,got everything as mentioned but 

In [6]:
def generate_category_ground_truth(df):
    """
    Creates a mapping of product_id -> list of other product_ids 
    in the same exact category.
    """
    ground_truth = {}
    # Group by the category string
    category_groups = df.groupby('category')['product_id'].apply(list)
    
    for category, products in category_groups.items():
        for i, target_id in enumerate(products):
            # All other products in this category are 'relevant'
            relevant = [p for j, p in enumerate(products) if i != j]
            if relevant:
                ground_truth[target_id] = relevant
                
    return ground_truth

In [7]:
import time

def evaluate_recommender(test_data, get_rec_func, K=5):
    """
    Evaluates the recommender using Precision@K, MRR, NDCG, and Latency.
    - test_data: Dict where key is product_id and value is list of relevant products.
    - get_rec_func: Your recommendation function.
    """
    precisions, reciprocal_ranks, ndcgs, latencies = [], [], [], []

    for target_id, relevant_ids in test_data.items():
        # Measure latency
        start = time.time()
        recs = get_rec_func(df, target_id, N=K)
        latency = time.time() - start
        latencies.append(latency)

        if hasattr(recs, 'product_id'):
            recommended_ids = recs['product_id'].tolist()
        else:
            recommended_ids = [r['product_id'] for r in recs]

        # Precision@K
        hits = len(set(recommended_ids) & set(relevant_ids))
        precisions.append(hits / K)

        # MRR
        rank = 0
        for i, rid in enumerate(recommended_ids):
            if rid in relevant_ids:
                rank = i + 1
                break
        reciprocal_ranks.append(1 / rank if rank > 0 else 0)

        # NDCG
        y_true = np.array([[1 if rid in relevant_ids else 0 for rid in recommended_ids]])
        y_score = np.array([[K - i for i in range(len(recommended_ids))]])
        try:
            if np.sum(y_true) > 1:
                ndcgs.append(ndcg_score(y_true, y_score, k=K))
            else:
                ndcgs.append(0.0)
        except Exception:
            ndcgs.append(0.0)

    return {
        f"Precision@{K}": np.mean(precisions),
        "MRR": np.mean(reciprocal_ranks),
        f"NDCG@{K}": np.mean(ndcgs),
        "Latency (s)": np.mean(latencies)
    }

In [8]:
import numpy as np
import pandas as pd
import random
from model_training import get_weighted_hybrid_recommendations

def randomized_weight_search(df, test_data, n_iter=50, K=5):
    """
    Randomly samples weight combinations to find the best configuration.
    Note: These keys must match the keys used in recs_to_combine 
    inside model_training.py exactly.
    """
    # Fixed keys to match your model_training.py Line 1526-1531
    # Update the keys in your randomized_weight_search function
    model_keys = [
    'basic_cosine', 'pca_features', 'content_tfidf', 'content_pca', 
    'review_text', 'sentiment', 'topic_lda', 'reviewer_overlap', 
    'knn_numeric', 'svd_collaborative'
    ]
    
    results = []
    best_score = -1
    best_weights = None

    print(f"Starting randomized search for {n_iter} iterations...")

    for i in range(n_iter):
        # 1. Generate random weights that sum to 1.0
        raw_weights = np.random.dirichlet(np.ones(len(model_keys)), size=1)[0]
        current_weights = dict(zip(model_keys, raw_weights))

        # 2. Define a wrapper for the evaluator
        def hybrid_func(d, pid, N):
            # Pass the correctly named weights to the hybrid function
            return get_weighted_hybrid_recommendations(d, pid, N=N, weights=current_weights)

        # 3. Evaluate using the category-based truth
        scores = evaluate_recommender(test_data, hybrid_func, K=K)
        score = scores[f'NDCG@{K}']
        
        results.append({'weights': current_weights, 'ndcg': score})

        if score > best_score:
            best_score = score
            best_weights = current_weights
            print(f"Iteration {i}: New Best NDCG@{K} = {best_score:.4f}")

    return best_weights, best_score

# --- Run the Optimization ---
category_truth = generate_category_ground_truth(df)
sample_truth = dict(list(category_truth.items())[:20]) 

optimal_weights, top_score = randomized_weight_search(df, sample_truth, n_iter=30)

print("\n--- OPTIMAL HYBRID WEIGHTS ---")
for k, v in optimal_weights.items():
    print(f"{k:20}: {v:.4f}")

Starting randomized search for 30 iterations...
Iteration 0: New Best NDCG@5 = 0.1983
Iteration 1: New Best NDCG@5 = 0.3691
Iteration 7: New Best NDCG@5 = 0.3983
Iteration 9: New Best NDCG@5 = 0.4000


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import time
import numpy as np

# 1. Generate the new Hybrid Ground Truth
print("Generating Hybrid Ground Truth...")
hybrid_truth = generate_category_ground_truth(df)
# Sample for evaluation speed
eval_sample = dict(list(hybrid_truth.items())[:30]) 

# 2. Define the models to test
# We use lambda wrappers to ensure all functions follow the same (product_id, N) signature
test_methods = {
    "Basic Cosine": lambda pid, N: get_recommendations(pid, N=N),
    "PCA Features": lambda pid, N: get_recommendations_with_pca(pid, N=N),
    "Content (TF-IDF)": lambda pid, N: get_content_recommendations(pid, N=N),
    "Content + PCA": lambda pid, N: get_content_recommendations_pca(pid, N=N),
    "Review Text": lambda pid, N: get_review_recommendations(pid, N=N),
    "Sentiment": lambda pid, N: get_sentiment_recommendations(pid, N=N),
    "Topic Modeling": lambda pid, N: get_topic_recommendations(pid, N=N),
    "Reviewer Overlap": lambda pid, N: get_reviewer_overlap_recommendations(pid, N=N),
    "SVD (Collab)": lambda pid, N: svd_product_recommendations(df, pid, n=N)
    #"OPTIMIZED HYBRID": lambda pid, N: get_weighted_hybrid_recommendations(pid, N=N, weights=optimal_weights)
}

# 3. Run Evaluation Loop
results = []
K_VAL = 5

for name, func in test_methods.items():
    print(f"Evaluating {name}...")
    # Using your existing evaluate_recommender logic
    # Note: Ensure evaluate_recommender is updated to handle (pid, N) signature
    metrics = evaluate_recommender(eval_sample, func, K=K_VAL)
    metrics['Model'] = name
    results.append(metrics)

# 4. Format and Display Results
kpi_df = pd.DataFrame(results)
# Organize columns for clarity
kpi_df = kpi_df[['Model', f'NDCG@{K_VAL}', 'MRR', f'Precision@{K_VAL}', 'Latency (s)']]
kpi_df = kpi_df.sort_values(by=f'NDCG@{K_VAL}', ascending=False).reset_index(drop=True)

print("\n--- FINAL KPI REPORT ---")
display(kpi_df)